In [ ]:
import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')


import seaborn as sns
sns.set()

from sklearn.metrics import classification_report, confusion_matrix 


In [ ]:
#df = pd.read_csv('NCDB_2016.csv', engine = 'python')
df = pd.read_csv('data01_simple.csv', engine = 'python')
#df = pd.read_csv('data01_simple_100.csv', engine = 'python')
df

In [ ]:
df.isnull().sum().sum()

In [ ]:
df[df.index.astype('str').str.contains('[^0-9]')].sum().sum()

In [ ]:
df_cat = df.astype('category').copy()


In [ ]:
df_int = df.astype('int').copy()

In [ ]:
df_cat_n = df_cat[:100000]

In [ ]:
print(df.columns)

In [ ]:
## Split Data into each class

In [ ]:
type(df_cat['P_ISEV'])
df_cat['P_ISEV']

# fatal
is_fatal =  df_cat['P_ISEV']==3
#print(is_fatal)
is_fatal_count = is_fatal.sum()
print(is_fatal_count)
df_cat_fatal = df_cat[is_fatal]
display(df_cat_fatal)


In [ ]:
# injury
is_injury =  df_cat['P_ISEV']==2
#print(is_injury)
is_injury_count = is_injury.sum()
print(is_injury_count)
df_cat_injury = df_cat[is_injury]
display(df_cat_injury)


In [ ]:
# non_injury
is_safe =  df_cat['P_ISEV']==1
#print(is_safe)
is_safe_count = is_safe.sum()
print(is_safe_count)
df_cat_safe = df_cat[is_safe]
display(df_cat_safe)

In [ ]:
# get the size of fatal datafram
min_size = df_cat_fatal.index.size
print(min_size)

In [ ]:
# get size of injury
df_cat_safe.index.size

In [ ]:
# size of injury
df_cat_injury.index.size

In [ ]:
# randomly sample n number of injury and no injury and append to fatal
df_cat_injury_select = df_cat_injury.sample(n=min_size)
df_cat_safe_select = df_cat_safe.sample(n=min_size)


In [ ]:
df_cat_injury_select.shape

In [ ]:
df_cat_safe_select.shape

In [ ]:
#concat the three dataframes
df_final = pd.concat([df_cat_fatal, df_cat_injury_select, df_cat_safe_select])
df_final.shape

In [ ]:
df_final.P_ISEV.unique()

In [ ]:
#Split between data and class

Y = df_final[df_cat.columns[-1]]
X = df_final[df_cat.columns[0:df_cat.columns.size -1]]
X

In [ ]:
display(X, Y)

## Feature selection using Random Forest

### Feature Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
forest.fit(X, Y)

### Get the inportant features from random forest

In [ ]:
importFeatures = forest.feature_importances_

### List the features by importancce

In [ ]:
importFeatures

In [ ]:
indices = np.argsort(importFeatures)[::-1]
print(indices)
featureLabel = X.columns[0:]
print(featureLabel)
rankedFeature = []
for f in range(X.shape[1]):
    rankedFeature.append(featureLabel[indices[f]])
    print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
print(rankedFeature)

### Reduce the number of features

In [ ]:
#select features that contribute more than 0.05
#[df_cat.columns[0:df_cat.columns.size -1]]
X_Selected = X[rankedFeature[0:10]]
display(X_Selected)
X_Selected.shape

#sprint into train and test 70/30
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

In [ ]:
#sprint into train and test 70/30
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
display(X_test, Y_test)
display(X_train, Y_train)

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000, random_state=0)
lr.fit(X_train, Y_train)

In [ ]:
y_pred = lr.predict(X_test)
#display(y_pred)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

In [ ]:
# print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
lr.intercept_

In [ ]:
# print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
lr.coef_

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

In [ ]:
# with L1 regularization
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1', C=1000, random_state=0)
lr.fit(X_train, Y_train)

In [ ]:
y_pred = lr.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### Support Vector Machines

In [ ]:
#from sklearn.svm import SVC
#svm = SVC(kernel='linear', C=1, random_state=0)
#svm.fit(X_train, Y_train)

In [ ]:
#y_pred = svm.predict(X_test)
#print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
#print('Accuracy of SVM regression classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))

In [ ]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix = confusion_matrix(Y_test, y_pred)
#print(confusion_matrix)
#print(classification_report(Y_test,y_pred))

### Decison Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy',max_depth=5, random_state=0)
tree.fit(X_train, Y_train)

In [ ]:
y_pred = tree.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0, n_jobs=2)
forest.fit(X_train, Y_train)

In [ ]:
y_pred = forest.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### K-N-N

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train, Y_train)

In [ ]:
y_pred = knn.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)
print(classification_report(Y_test,y_pred))

### SVM

In [ ]:
from sklearn.svm import SVC
#svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
#svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)

In [ ]:
#svm.fit(X_train, Y_train)

In [ ]:
#y_pred = svm.predict(X_test)
#print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
#print('Accuracy of SVM regression classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))

In [ ]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix = confusion_matrix(Y_test, y_pred)
#print(confusion_matrix)

In [ ]:
'''
check sigmoid and rbf
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
clf = BaggingClassifier(SVC(C=1.0,
        cache_size=200,
        class_weight=None,
        coef0=0.0,
        decision_function_shape=None,
        degree=3,
        gamma='auto',
        kernel='linear',
        max_iter=-1,
        probability=False,
        random_state=None,
        shrinking=True,
        tol=0.001,
        verbose=False,
        ))
'''

In [ ]:
#clf.fit(X_train, Y_train)